In [61]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.metrics.pairwise import cosine_similarity

pd.options.display.max_rows = 10
pd.options.display.max_columns = 15

#guest_id = int(input('insert a user id : '))
guest_id = 541
#data[data[guest_id] > 0][guest_id].to_frame('user id')

In [62]:
data = pd.read_table('data/u.data', sep='\t', usecols=['user id', 'movie id', 'rating'])
data

,user id,movie id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [63]:
data = data.pivot_table(index='movie id', columns='user id', values='rating', fill_value=0)
data

user id,1,2,3,4,5,6,7,...,937,938,939,940,941,942,943
movie id,,,,,,,,,,,,,,,
1,5,4,0,0,4,4,0,...,0,4,0,0,5,0,0
2,3,0,0,0,3,0,0,...,0,0,0,0,0,0,5
3,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,3,0,0,0,0,0,5,...,0,0,0,2,0,0,0
5,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1680,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0


In [64]:
sim = DataFrame()
for i in data.columns:
    sim[i] = DataFrame(cosine_similarity([data[guest_id]], [data[i]]))

top10_near = sim.T.sort_values(by=0, ascending=False).head(11).T
top10_near

,541,798,712,807,290,642,70,42,588,705,885
0,1.0,0.47885,0.476921,0.459135,0.456252,0.456122,0.45466,0.448914,0.430938,0.429087,0.427202


In [65]:
mult_rec = DataFrame()
for i in top10_near.columns:
    if i != guest_id:
        mult_rec[i] = data[i] * (1 + top10_near[i][0])

data = mult_rec.T.sum().sort_values(ascending=False).to_frame('recommend rate').head(10)
data

,recommend rate
movie id,
181,68.281331
174,68.249672
210,66.766814
172,65.398969
143,65.396972
50,65.375324
82,63.865383
588,62.462061
95,62.456202


In [66]:
item = pd.read_table('data/u.item', sep='|', usecols=['movie id', 'movie title', 'release date'])
item

,movie id,movie title,release date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994


In [67]:
item = item.set_index(['movie id'])
item

,movie title,release date
movie id,,
1,Toy Story (1995),01-Jan-1995
2,GoldenEye (1995),01-Jan-1995
3,Four Rooms (1995),01-Jan-1995
4,Get Shorty (1995),01-Jan-1995
5,Copycat (1995),01-Jan-1995
...,...,...
1678,Mat' i syn (1997),06-Feb-1998
1679,B. Monkey (1998),06-Feb-1998
1680,Sliding Doors (1998),01-Jan-1998


In [68]:
pd.concat([data, item], axis=1, join_axes=[data.index])

,recommend rate,movie title,release date
movie id,,,
181,68.281331,Return of the Jedi (1983),14-Mar-1997
174,68.249672,Raiders of the Lost Ark (1981),01-Jan-1981
210,66.766814,Indiana Jones and the Last Crusade (1989),01-Jan-1989
172,65.398969,"Empire Strikes Back, The (1980)",01-Jan-1980
143,65.396972,"Sound of Music, The (1965)",01-Jan-1965
50,65.375324,Star Wars (1977),01-Jan-1977
82,63.865383,Jurassic Park (1993),01-Jan-1993
588,62.462061,Beauty and the Beast (1991),01-Jan-1991
95,62.456202,Aladdin (1992),01-Jan-1992
